# Fugue with Polars (UNDER DEVELOPMENT)

Have questions? Chat with us on Github or Slack:

[![Homepage](https://img.shields.io/badge/fugue-source--code-red?logo=github)](https://github.com/fugue-project/fugue)
[![Slack Status](https://img.shields.io/badge/slack-join_chat-white.svg?logo=slack&style=social)](http://slack.fugue.ai)

Polars is a Rust-based DataFrame library that supports multi-threaded and out-of-core operations. The performance of Polars is already very good on a local machine, so the focus of the Fugue-Polars integration is scaling out to a cluster. Fugue also has [FugueSQL](../../quick_look/ten_minutes_sql.ipynb) to run SQL on top of DataFrames, but it is a lower priority for Polars because of the existing [DuckDB](../../integrations/backends/duckdb.ipynb) integration that can be used pretty easily with Polars. Because both are based on Apache Arrow, they can be used together with zero-copy.

There are two main scenarios:

* Users may have a Spark, Dask, or Ray cluster available that they want to utilize to speed up an operation. Polars can handle out-of-core operations well, but it can still be faster to execute on top of a cluster.
* Users can use the Fugue API and develop locally on Polars. When ready to scale, users just need to change the backend. 

Note that for distributed computing operations, a big overhead comes from I/O. This means that even if the data operation executes quickly, the transfer may make total execution time longer.

## Running Polars Code Distributedly



In [9]:
import polars as pl
from fugue import transform
data = {"group": ["A", "A", "A", "B", "B", "B"],
        "number": [10, 20, 30, 15, 25, 35]}
df = pl.DataFrame(data)

def tr1(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(pl.col("number").diff().alias("diff"))

transform(df, tr1, schema="group:str,number:int,diff:float", partition={"by": "group"})

_0 _State.RUNNING -> _State.FAILED  Schema can't be empty


SchemaError: Schema can't be empty

In [ ]:
    def tr2(dfs: Iterable[pl.DataFrame]) -> Iterator[pl.DataFrame]:
        for df in dfs:
            tdf = df.with_column(pl.lit(1, pl.Int32()).alias("c"))
            yield tdf